_What we want to do is build up a dictionary of N-grams, which are pairs, triplets or more (the N) of words that pop up in the training data, with the value being the number of times they showed up. After we have this dictionary, as a naive example we could actually predict sentences by just randomly choosing words within this dictionary and doing a weighted random sample of the connected words that are part of n-grams within the keys._

_Lets see how far we can get with N-grams without outside resources._

_We have a text file for [Pride and Prejudice from Project Gutenberg](https://www.gutenberg.org/ebooks/1342) stored as pg1342.txt in the same folder as our notebook, but also available online directly. Let's load the text to a string since it's only 701KB, which will fit in memory nowadays._

_**Note** : If we wanted to be more memory efficient we should parse the text file on a line or character by character basis, storing as needed, etc._

In [1]:
# Find the number links by looking on Project Gutenberg in the address bar for a book.
books = {'Pride and Prejudice': '1342',
         'Huckleberry Fin': '76',
         'Sherlock Holmes': '1661'}

book = books['Pride and Prejudice']

f = open('gutenberg.txt', encoding='utf-8')
txt = f.read()
f.close()

# See the number of characters and the first 50 characters to confirm it is there    
print(len(txt), ',', txt[:50] , '...')

775714 , The Project Gutenberg eBook of Pride and Prejudice ...


Great, now lets split into words into a big list, splitting on anything non-alphanumeric [A-Za-z0-9] (as well as punctuation) and forcing everything lowercase:

In [2]:
import re
words = re.split('[^A-Za-z0-9]+', txt.lower())
words = list(filter(None, words)) # Remove empty strings

# Print length of list
print(len(words))

126262


## N-grams generation
From this we can now generate N-grams, lets start with a 1-gram, basically the set of all the words

**Note** : One could use a dictionary instead of a set and keeping count of the occurances gives word frequency

In [3]:
# Create set of all unique words, this throws away any information about frequency however
gram1 = set(words)

print(len(gram1))

# Print 20 elements of the set only
print(list(gram1)[:20])

6603
['atoned', 'grosvenor', 'tremble', 'private', 'understood', 'painfully', 'read', 'selves', 'detestable', 'nonproprietary', 'bitterest', 'persisting', 'couple', 'useless', 'fashionable', 'inducements', 'stopping', 'pressingly', 'staggered', 'powerful']


Lets try and get the 2-gram now, which is pairs of words. Let's have a quick look to see the last 10 and how they look.

In [4]:
# See the last 10 pairs
for i in range(len(words)-10, len(words)-1):
    print(words[i], words[i+1])

subscribe to
to our
our email
email newsletter
newsletter to
to hear
hear about
about new
new ebooks


Okay, seems good, lets get all word pairs, and then generate a set of unique pairs from it

In [5]:
word_pairs = [(words[i], words[i+1]) for i in range(len(words)-1)]
print(len(word_pairs))

gram2 = set(word_pairs)
print(len(gram2))

# Print 20 elements from gram2
print(list(gram2)[:20])

126261
55852
[('ones', 'paid'), ('objection', 'said'), ('information', 'however'), ('is', 'obliged'), ('own', 'opinion'), ('her', 'going'), ('amiable', 'appearance'), ('awake', 'to'), ('encouraged', 'to'), ('winter', 'you'), ('often', 'think'), ('by', 'being'), ('speedily', 'pronounced'), ('had', 'detected'), ('in', 'half'), ('intermarriage', 'she'), ('perseveringly', 'by'), ('was', 'warmly'), ('first', 'was'), ('his', 'unpardonable')]


## N-Grams Frequency
Okay, that was fun, but this isn't enough, we need frequency if we want to have any sense of probabilities, which is what N-grams are about. Instead of using sets, lets create a dictionary with counts

In [6]:
gram1 = dict()

# Populate 1-gram dictionary
for word in words:
    if word in gram1:
        gram1[word] += 1
    else:
        gram1[word] = 1 # Start a new entry with 1 count since saw it for the first time.

# Turn into a list of (word, count) sorted by count from most to least
gram1 = sorted(gram1.items(), key=lambda item: -item[1])

# Print top 20 most frequent words
print([(word, freq) for word, freq in gram1[:20]])

[('the', 4520), ('to', 4245), ('of', 3736), ('and', 3657), ('her', 2225), ('i', 2070), ('a', 2007), ('in', 1941), ('was', 1848), ('she', 1710), ('that', 1595), ('it', 1550), ('not', 1457), ('you', 1433), ('he', 1339), ('his', 1271), ('be', 1260), ('as', 1192), ('had', 1177), ('with', 1100)]


For Pride and Prejudice, the words 'the', 'to', 'of', and 'and' were the top four most common words. Sounds about right, not too interesting yet, lets see what happens with 2-grams.

In [7]:
gram2 = dict()

# Populate 2-gram dictionary
for i in range(len(words)-1):
    key = (words[i], words[i+1])
    if key in gram2:
        gram2[key] += 1
    else:
        gram2[key] = 1

# Turn into a list of (word, count) sorted by count from most to least
gram2 = sorted(gram2.items(), key=lambda item: -item[1])

# Print top 20 most frequent words
print([(word, freq) for word, freq in gram2[:20]])

[(('of', 'the'), 498), (('to', 'be'), 445), (('in', 'the'), 401), (('i', 'am'), 303), (('mr', 'darcy'), 273), (('to', 'the'), 268), (('of', 'her'), 262), (('it', 'was'), 251), (('of', 'his'), 235), (('she', 'was'), 212), (('she', 'had'), 205), (('had', 'been'), 199), (('it', 'is'), 194), (('i', 'have'), 188), (('to', 'her'), 179), (('that', 'he'), 177), (('could', 'not'), 167), (('he', 'had'), 166), (('and', 'the'), 165), (('for', 'the'), 162)]


It looks like "of the" and "to be" are the top two most common 2-grams, sounds good.

## Next word prediction

What can we do with this? Well lets see what happens if we take a random word from all the words, and build a sentence by just choosing the most common pair that has that word as it's start.

In [8]:
start_word = words[int(len(words)/4)]
print(start_word)

after


I just went ahead and chose the word that appears $1/4$ of the way into words, random enough.

Now in a loop, iterate through the frequency list (most frequent first) and see if it matches the first word in a pair, if so, the next word is the second element in the word pair, and continue with that word. Stop after N words or the list does not contain that word.

**Note** : gram2 is a list that contains (key,value) where key is a word pair (first, second),
           so you need element[0][0] for first word and element [0][1] for second word

In [9]:
def get2GramSentence(word, n = 50):
    words = []
    for i in range(n):
        words.append(word)
        # Find Next word
        word = next((element[0][1] for element in gram2 if element[0][0] == word), None)
        if not word:
            break
    return ' '.join(words)

word = start_word
print("Start word:", word)

print('2-gram sentence: "', get2GramSentence(word, 20), '"')

Start word: after
2-gram sentence: " after a very much as to be so much as to be so much as to be so much as "


It gets stuck in a loop pretty much straight away. Not very interesting, try out other words and see what happens.

In [10]:
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print("Start word:", word)
    print('2-gram sentence: "', get2GramSentence(word, 20), '"')

Start word: and
2-gram sentence: " and the whole party were to be so much as to be so much as to be so much as "
Start word: he
2-gram sentence: " he had been so much as to be so much as to be so much as to be so much "
Start word: she
2-gram sentence: " she was not be so much as to be so much as to be so much as to be so "
Start word: when
2-gram sentence: " when she was not be so much as to be so much as to be so much as to be "
Start word: john
2-gram sentence: " john with the whole party were to be so much as to be so much as to be so much "
Start word: never
2-gram sentence: " never be so much as to be so much as to be so much as to be so much as "
Start word: i
2-gram sentence: " i am sure i am sure i am sure i am sure i am sure i am sure i am "
Start word: how
2-gram sentence: " how much as to be so much as to be so much as to be so much as to be "


## Weighted random choice based on frequency

**This is our simple probabilistic MLE N-gram model**

Same thing. Okay, lets randomly choose from the subset of all 2grams that matches the first word, using a weighted-probability based on counts.

In [11]:
import random
def weighted_choice(choices):
    total = sum(w for c, w in choices)
    r = random.uniform(0, total)
    upto = 0
    for c, w in choices:
        if upto + w > r:
            return c
        upto += w
    
def get2GramSentenceRandom(word, n = 50):
    words = []
    for i in range(n):
        words.append(word)
        # Get all possible elements ((first word, second word), frequency)
        choices = [element for element in gram2 if element[0][0] == word]
        if not choices:
            break
        
        # Choose a pair with weighted probability from the choice list
        word = weighted_choice(choices)[1]
    return ' '.join(words)

In [12]:
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print("Start word:", word)
    print('2-gram sentence: "', get2GramSentenceRandom(word, 20), '"')

Start word: and
2-gram sentence: " and could not take any other about gracechurch street and assistance they are talking to have jane s disposition greater "
Start word: he
2-gram sentence: " he chose to her life it to jane s approach was now and her to himself did it was an "
Start word: she
2-gram sentence: " she cared i might bring was in it with offers of a salad and lydia s quick where i had "
Start word: when
2-gram sentence: " when you were necessary business was told of questions to him alone mary and they were grieved she thought it "
Start word: john
2-gram sentence: " john with colonel forster instantly passed together and as to refrain from any of the longbourn and they may lead "
Start word: never
2-gram sentence: " never happened in her husband her stay quietly answered me more easily with a manner you to be explicit let "
Start word: i
2-gram sentence: " i thought she called during dinner to leave town dear wickham for the sisters came forward and one of friends "
Start 

Now that's way more interesting! Those are starting to look like sentences!

Let's try a longer sentence

In [13]:
word = 'it'
print("Start word:", word)
print('2-gram sentence: "', get2GramSentenceRandom(word, 50), '"')

Start word: it
2-gram sentence: " it earlier i can afford some passages which we begin freely available with his memory of schemes but it is gone jane in evident to hold her maternal feelings which her air of my dear said mrs bennet s marrying and sensible of the trouble of it her sister said "


Pretty cool, lets see what happens when we go to N-grams above 2.

## Tri-grams and more
Okay, let's create a Ngram generator that can let us make ngrams of arbitrary sizes

In [14]:
def generateNgram(n=1):
    gram = dict()
    
    # Some helpers to keep us crashing the PC for now
    assert n > 0 and n < 100
    
    # Populate N-gram dictionary
    for i in range(len(words)-(n-1)):
        key = tuple(words[i:i+n])
        if key in gram:
            gram[key] += 1
        else:
            gram[key] = 1

    # Turn into a list of (word, count) sorted by count from most to least
    gram = sorted(gram.items(), key=lambda item: -item[1])
    return gram

trigram = generateNgram(3)
# Print top 20 most frequent ngrams
print(trigram[:20])

[(('i', 'do', 'not'), 62), (('i', 'am', 'sure'), 62), (('project', 'gutenberg', 'tm'), 57), (('as', 'soon', 'as'), 55), (('she', 'could', 'not'), 50), (('that', 'he', 'had'), 37), (('it', 'would', 'be'), 34), (('in', 'the', 'world'), 34), (('the', 'project', 'gutenberg'), 33), (('i', 'am', 'not'), 32), (('i', 'dare', 'say'), 31), (('it', 'was', 'not'), 30), (('could', 'not', 'be'), 30), (('that', 'he', 'was'), 29), (('mr', 'darcy', 's'), 29), (('that', 'it', 'was'), 28), (('on', 'the', 'subject'), 28), (('of', 'mr', 'darcy'), 27), (('would', 'have', 'been'), 27), (('as', 'well', 'as'), 27)]


In [15]:
def getNGramSentenceRandom(gram, word, n = 50):
    words = []
    for i in range(n):
        words.append(word)
        # Get all possible elements ((first word, second word), frequency)
        choices = [element for element in gram if element[0][0] == word]
        if not choices:
            break
        
        # Choose a pair with weighted probability from the choice list
        word = weighted_choice(choices)[1]
    return ' '.join(words)

for n in range(2,10):
    # Generate ngram list
    print(f"Generating {n}-gram list...")
    ngram = generateNgram(n)
    print("Done")
    
    # Try out a bunch of sentences
    for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
        print("Start word:", word)
        print(f'{n}-gram sentence: \n"{getNGramSentenceRandom(ngram, word, 20)}"')
        print()
        
    print('***************************************************************************')

Generating 2-gram list...
Done
Start word: and
2-gram sentence: 
"and down for caprice that she answered only let her friend and taking her friend says that head over the"

Start word: he
2-gram sentence: 
"he scarcely spoke joined them both of his determination i think it takes a most tractable creatures if i can"

Start word: she
2-gram sentence: 
"she wrote must date june for the monthly balls will you find that can you know what do cure the"

Start word: when
2-gram sentence: 
"when so but against mr collins having longbourn elizabeth had always so soon went away before i was nothing further"

Start word: john
2-gram sentence: 
"john told you in love it and even sir william was decided opinion in his father can get together very"

Start word: never
2-gram sentence: 
"never make my family from pressing and dazzling with some news well now employment and myself the short and i"

Start word: i
2-gram sentence: 
"i shall certainly spare half an exertion on his character she had chosen 

8-gram sentence: 
"never without a spot or to add that we may only imagine me when it she should infinitely better that"

Start word: i
8-gram sentence: 
"i do you hold out as she has a second daughter elizabeth who that had ruined the way in spite"

Start word: how
8-gram sentence: 
"how i cannot wonder who must be very part of what congratulations to be placed himself to see if not"

***************************************************************************
Generating 9-gram list...
Done
Start word: and
9-gram sentence: 
"and if he was consequently was hurt he offered to one seemed to have two hours giving her mind that"

Start word: he
9-gram sentence: 
"he was the room elizabeth said in this ebook pride which reflection in their father would be alone his plan"

Start word: she
9-gram sentence: 
"she was short i am growing more like your admiration and i wonder and backed by design nonsense of him"

Start word: when
9-gram sentence: 
"when he chooses nobody else i have got some cau

The sentences produced by higher-level N-gram looks almost like normal sentences if you squint a little!

## Additive Smoothin

Let's try and adjust our current function of create n_grams with laplace smoothing.

In [36]:
# Create 4 gram with laplace smoothing 

def create_gram_laplace(vocab, n=1):
    gram_dict = dict()
    
    assert n > 0 and n < 25 # keep assertions and safety more strictly
    
    for i in range(len(vocab) - n - 1):
        gram = tuple(vocab[i:i+n])
        candidate = gram_dict.get(gram, 1)
        gram_dict[gram] = candidate + 1
        
    sorted_gram_dict = sorted(gram_dict.items(), key=lambda item: -item[1])
    return sorted_gram_dict

In [37]:
four_gram = create_gram_laplace(words, 4)

In [38]:
four_gram[:20]

[(('i', 'do', 'not', 'know'), 20),
 (('project', 'gutenberg', 'tm', 'electronic'), 19),
 (('at', 'the', 'same', 'time'), 17),
 (('the', 'rest', 'of', 'the'), 16),
 (('i', 'am', 'sure', 'i'), 16),
 (('in', 'the', 'course', 'of'), 15),
 (('lady', 'catherine', 'de', 'bourgh'), 15),
 (('as', 'soon', 'as', 'they'), 14),
 (('her', 'uncle', 'and', 'aunt'), 14),
 (('mr', 'and', 'mrs', 'gardiner'), 14),
 (('the', 'project', 'gutenberg', 'tm'), 14),
 (('the', 'project', 'gutenberg', 'literary'), 14),
 (('project', 'gutenberg', 'literary', 'archive'), 14),
 (('gutenberg', 'literary', 'archive', 'foundation'), 14),
 (('gutenberg', 'tm', 'electronic', 'works'), 13),
 (('if', 'you', 'do', 'not'), 12),
 (('in', 'the', 'united', 'states'), 11),
 (('of', 'the', 'project', 'gutenberg'), 11),
 (('for', 'the', 'sake', 'of'), 11),
 (('as', 'soon', 'as', 'he'), 10)]

In [39]:
# we should have at least 2 occurences to say the least
four_gram[:-20:-1]

[(('newsletter', 'to', 'hear', 'about'), 2),
 (('email', 'newsletter', 'to', 'hear'), 2),
 (('our', 'email', 'newsletter', 'to'), 2),
 (('to', 'our', 'email', 'newsletter'), 2),
 (('subscribe', 'to', 'our', 'email'), 2),
 (('to', 'subscribe', 'to', 'our'), 2),
 (('how', 'to', 'subscribe', 'to'), 2),
 (('and', 'how', 'to', 'subscribe'), 2),
 (('ebooks', 'and', 'how', 'to'), 2),
 (('new', 'ebooks', 'and', 'how'), 2),
 (('our', 'new', 'ebooks', 'and'), 2),
 (('produce', 'our', 'new', 'ebooks'), 2),
 (('help', 'produce', 'our', 'new'), 2),
 (('to', 'help', 'produce', 'our'), 2),
 (('how', 'to', 'help', 'produce'), 2),
 (('foundation', 'how', 'to', 'help'), 2),
 (('archive', 'foundation', 'how', 'to'), 2),
 (('literary', 'archive', 'foundation', 'how'), 2),
 (('make', 'donations', 'to', 'the'), 2)]

In [40]:
# let's see what kind of sentences we can generate with these 4-gram we ve got

for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
        print("Start word:", word)
        print(f'{4}-gram sentence: \n"{getNGramSentenceRandom(four_gram, word, 20)}"')
        print()

Start word: and
4-gram sentence: 
"and determining probabilities and our dear jane her daughter endeavoured to have been a mother up nonproprietary or 1 e"

Start word: he
4-gram sentence: 
"he was going away without being so there is adjusting her sister s delight to him before you were obliged"

Start word: she
4-gram sentence: 
"she rambled about him and amiable charming daughter let down and asking me when the wife yes my sentiments you"

Start word: when
4-gram sentence: 
"when i think meanly of its injustice of my other alteration for the evening elizabeth where she who followed her"

Start word: john
4-gram sentence: 
"john with a peep at the valley as much affection will not engross her again with her eye power to"

Start word: never
4-gram sentence: 
"never resent the least more interesting intelligence of doing a mixture of the moral if you reasons to miss bennet"

Start word: i
4-gram sentence: 
"i talked of distinguishing the whole evening but charlotte says that jane which